In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd, matplotlib.pyplot as plt

from hyperopt import hp, tpe, fmin, Trials

from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split

import lightgbm as lgb

import os, gc, random, time, pickle, json
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-cal-mod2/calendar_mod_2.csv


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Load data

In [3]:
def cal_transform(df):
#     sc = MinMaxScaler(feature_range=(0, 1))
    sc = StandardScaler()
    cols = ['wholesale_inventory','wholesale_sales','retai_t_f_sales','urban_cpi','cpi','unemp_rate','ppi','gdp']
    df_scaled = sc.fit_transform(df[cols])
    df_scaled = pd.DataFrame(df_scaled)
    for i, col in enumerate(cols):
        df[col] = df_scaled[i]
    return df

In [4]:
path = "../input/m5-forecasting-accuracy"

calendar = pd.read_csv(os.path.join('/kaggle/input/m5-cal-mod2/calendar_mod_2.csv'))
calendar = cal_transform(calendar).pipe(reduce_mem_usage)
print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
selling_prices = pd.read_csv(os.path.join(path, "sell_prices.csv"))
print('selling_prices has {} rows and {} columns'.format(selling_prices.shape[0], selling_prices.shape[1]))
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
print('sample_submission has {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
sales = pd.read_csv(os.path.join(path, "sales_train_validation.csv"))
print('sales has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))

Mem. usage decreased to  0.15 Mb (54.0% reduction)
Calendar has 1969 rows and 22 columns
selling_prices has 6841121 rows and 4 columns
sample_submission has 60980 rows and 29 columns
sales has 30490 rows and 1919 columns


## Prepare calendar data

In [5]:
def prep_calendar(df):
    # time features
    
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.week
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    df = df.drop(["date", "weekday", "event_type_1", "event_type_2"], axis=1)
    df = df.assign(d = df.d.str[2:].astype(int))
    to_ordinal = ["event_name_1", "event_name_2"] 
    df[to_ordinal] = df[to_ordinal].fillna("1")
    df[to_ordinal] = OrdinalEncoder(dtype="int").fit_transform(df[to_ordinal]) + 1
    to_int8 = ["wday", "month", "snap_CA", "snap_TX", "snap_WI"] + to_ordinal
    df[to_int8] = df[to_int8].astype("int8")
    
    return df

calendar = prep_calendar(calendar)
# calendar.head()

In [6]:
penalty = 1.15 
# Define custom asymmetric MSE loss functions since the penalty of incorrect prediction is not symmetric in regards to RMSE's relation to competition score WRMSSE
# Custom loss which gives [1.15] times more penalty when the true targets are more than predictions as compared to less

# Objective function (fobj)
# The training loss is the function that is optimized using gradient descent.
# The training loss typically needs to be a function that has a convex gradient (first derivative) and hessian (second derivative). 
# It is also preferably continuous, finite, and non-zero. The last one is important because sections where the function is zero can freeze gradient descent.
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * penalty)
    hess = np.where(residual < 0, 2, 2 * penalty)
    return grad, hess

# Evaluation metric (feval)
# The validation loss is used to tune hyper-parameters.
# The validation loss can be non-convex, non-differentiable, and discontinuous.
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * penalty) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [7]:
def optimize_parameters(df, max_evals=20, dept="FOODS_1"):
    # define fixed hyperparameters
    params = {
            'boosting_type': 'gbdt',  #goss, dart, gbdt
            'objective': 'tweedie',  #poisson, tweedie, rmse
            'tweedie_variance_power': 1.1,
            'metric': 'rmcustomse',
            'subsample': 0.5,
            'subsample_freq': 1,
            'learning_rate': 0.03,
            'feature_fraction': 0.5,
            'max_bin': 100,
            'n_estimators': 1400,
            'boost_from_average': False,
            'verbose': -1
            }
    
    # define floating hyperparameters
    space = {
        'max_depth': hp.quniform('max_depth', 3, 12, 1),
        'num_leaves': hp.quniform('num_leaves', 10, 2040, 20),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 10, 2040, 20),
            }
    
    # define the objective function to optimize the hyperparameters
    def objective(floating_params):
        params['max_depth'] = int(floating_params['max_depth'])
        params['num_leaves'] = int(floating_params['num_leaves'])
        params['min_data_in_leaf'] = int(floating_params['min_data_in_leaf'])
        print(params)
        regressor = lgb.LGBMRegressor(**params)
        
        x_sample = df.sample(int(df.shape[0]/50), random_state=SEED)
        x_train_sample, y_train_sample = x_sample[x], x_sample['demand']
        
        score = cross_val_score(regressor, x_train_sample, y_train_sample, cv=StratifiedKFold(),
                                scoring=make_scorer(mean_squared_error, greater_is_better=True)
                                ).mean()
        print("rmse {:.3f}".format(score))
        return score

    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=max_evals)
    
    params['num_iterations'] = num_iterations
    
    best['max_depth'] = int(best['max_depth'])
    best['num_leaves'] = int(best['num_leaves'])
    best['min_data_in_leaf'] = int(best['min_data_in_leaf'])
        
    with open(dept+'_params.txt', 'w') as file:
        file.write(json.dumps(params))
    
    return params

In [8]:
LAGS = [5, 7, 30]
WINDOWS = [7, 28, 56]
FIRST = 1914
LENGTH = 28
num_iterations = 2500 # num boosting cycles for LGBM
ESR = 150 # early_stopping_rounds
VE = 25 # verbose_eval
SEED = 66

def demand_features(df):
    """ Derive features from sales data and remove rows with missing values """
    for lag in LAGS:
        df[f'lag_t{lag}'] = df.groupby('id')['demand'].transform(lambda x: x.shift(lag)).astype("float32")
        for w in WINDOWS:
            df[f'rolling_mean_lag{lag}_w{w}'] = df.groupby('id')[f'lag_t{lag}'].transform(lambda x: x.rolling(w).mean()).astype("float32")
#             df[f'rolling_skew_{lag}_w{w}'] = df.groupby(["id"])[f'lag_t{lag}'].transform(lambda x: x.rolling(w).skew()).astype("float32")
    return df

def demand_features_eval(df):
    """ Same as demand_features but for the step-by-step evaluation """
    out = df.groupby('id', sort=False).last()
    for lag in LAGS:
        out[f'lag_t{lag}'] = df.groupby('id', sort=False)['demand'].nth(-lag-1).astype("float32")
        for w in WINDOWS:
            out[f'rolling_mean_lag{lag}_w{w}'] = df.groupby('id', sort=False)['demand'].nth(list(range(-lag-w, -lag))).groupby('id', sort=False).mean().astype("float32")
#             out[f'rolling_skew_{lag}_w{w}'] = df.groupby('id', sort=False)['demand'].nth(list(range(-lag-w, -lag))).groupby('id', sort=False).skew().astype("float32")
    return out.reset_index()

def prep_data(df, drop_d=1000, dept_id="FOODS_1"):
    """ Prepare model data sets """
    
    print(f"\nWorking on dept {dept_id}")
    # Filter on dept_id
    df = df[df.dept_id == dept_id]
    df = df.drop(["dept_id", "cat_id"], axis=1)
    
    # Kick out old dates
    df = df.drop(["d_" + str(i+1) for i in range(drop_d)], axis=1)

    # Reshape to long
    df = df.assign(id=df.id.str.replace("_validation", ""))
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(FIRST + i) for i in range(2 * LENGTH)])
    df = df.melt(id_vars=["id", "item_id", "store_id", "state_id"], var_name='d', value_name='demand')
    df = df.assign(d=df.d.str[2:].astype("int64"),
                   demand=df.demand.astype("float32"))
    
    # Add demand features
    df = demand_features(df)
    
    # Remove rows with NAs
    df = df[df.d > (drop_d + max(LAGS) + max(WINDOWS))]
 
    # Join calendar & prices
    df = df.merge(calendar, how="left", on="d")
    df = df.merge(selling_prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])
 
    # Add amount features
    df['amount'] = df['demand'] * df['sell_price']
#     gra = df.groupby(["store_id", "item_id"])["amount"]
#     df["amount_rel_diff"] = df.groupby(["store_id", "item_id"])["amount"].pct_change()
#     df['growing_amount'] = df['amount'] * (1 + df['amount_rel_diff']/100)
#     df["amount_cumrel"] = (gra.shift(0) - gra.cummin()) / (1 + gra.cummax() - gra.cummin())
    for lag in LAGS:
        df[f'amount_lag_t{lag}'] = df.groupby('id')['amount'].transform(lambda x: x.shift(lag)).astype("float32")
        for w in WINDOWS:
            df[f'rolling_mean_amount_lag{lag}_w{w}'] = df.groupby('id')[f'amount_lag_t{lag}'].transform(lambda x: x.rolling(w).mean()).astype("float32")
#             df[f'rolling_skew_amount_lag{lag}_w{w}'] = df.groupby('id')[f'amount_lag_t{lag}'].transform(lambda x: x.rolling(w).std()).astype("float32")
#     del gra
#     gc.collect()
    
    # Add price features
    df['lag_price_t1'] = df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    df['price_change_t1'] = (df['lag_price_t1'] - df['sell_price']) / (df['lag_price_t1'])
    df['rolling_price_max_t365'] = df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    df['price_change_t365'] = (df['rolling_price_max_t365'] - df['sell_price']) / (df['rolling_price_max_t365'])
    df['rolling_price_std_t7'] = df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    df['rolling_price_std_t28'] = df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(28).std())
    df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)

    # We can do some basic aggregations
    gr = df.groupby(["store_id", "item_id"])["sell_price"]
    df["sell_price_rel_diff"] = gr.pct_change()
    df['growing_price'] = df['sell_price'] * (1 + df['sell_price_rel_diff']/100)
#     df["sell_price_roll_sd7"] = gr.transform(lambda x: x.rolling(7).std())
#     df["sell_price_cumrel"] = (gr.shift(0) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    df['price_max'] = df.groupby(['store_id','item_id'])['sell_price'].transform('max')
    df['price_min'] = df.groupby(['store_id','item_id'])['sell_price'].transform('min')
    # and do price normalization (min/max scaling)
    df['price_norm'] = df['sell_price']/df['price_max']
    df.drop(['price_max'], inplace = True, axis = 1)
    df['price_std'] = df.groupby(['store_id','item_id'])['sell_price'].transform('std')
    df['price_mean'] = df.groupby(['store_id','item_id'])['sell_price'].transform('mean')
    del gr
    gc.collect()
    
    # Some items are can be inflation dependent
    # and some items are very "stable"
    df['price_nunique'] = df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
    df['item_nunique'] = df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

    # Now we can add price "momentum" (some sort of)
    # Shifted by week 
    # by month mean
    # by year mean
    df['price_momentum'] = df['sell_price']/df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
    df['price_momentum_m'] = df['sell_price']/df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
    df['price_momentum_y'] = df['sell_price']/df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')
    del df['month'], df['year']
    df = df.drop(["wm_yr_wk"], axis=1)

    # Ordinal encoding of remaining categorical fields
    for v in ["item_id", "store_id", "state_id"]:
        df[v] = OrdinalEncoder(dtype="int").fit_transform(df[[v]]).astype("int16") + 1
    
    # Determine list of covariables
    x = list(set(df.columns) - {'id', 'd', 'demand', 'amount','wm_yr_wk'})
            
    # Split into test, valid, train
    test = df[df.d >= FIRST - max(LAGS) - max(WINDOWS)]
    df = df[df.d < FIRST]

    xtrain, xvalid, ytrain, yvalid = train_test_split(df[x], df["demand"], test_size=0.2, shuffle=True, random_state=SEED) 
    
    train = lgb.Dataset(xtrain, label = ytrain)
    valid = lgb.Dataset(xvalid, label = yvalid)

    return train, valid, test, x, df

In [9]:
def fit_model(train, valid, dept, update_hyperparameters=False):
    """ Fit LightGBM model """
     
    if (update_hyperparameters and os.path.exists('/kaggle/working/'+dept+'_params.txt')):
        with open(dept+'_params.txt') as params_file:    
            params = json.load(params_file)
    else:
        params = optimize_parameters(df, dept=dept)
        

    fit = lgb.train(params, 
                    train, 
                    num_boost_round = num_iterations, 
                    valid_sets = [valid], 
                    early_stopping_rounds = ESR,
                    verbose_eval = VE, 
                    fobj=custom_asymmetric_train, 
                    feval=custom_asymmetric_valid
                   )
    
#     # feature names
#     print('Feature names:', fit.feature_name())

#     # feature importances
#     print('Feature importances:', list(fit.feature_importance()))

#     print('Plot Feature Importance...')
#     fig, ax = plt.subplots(figsize=(12,8))
#     lgb.plot_importance(fit, ax=ax, precision=0, height=0.8, max_num_features=35, figsize=(8,12), title=dept)
#     ax.grid(False)
#     plt.title("LightGBM - Feature Importance on dept " +dept, fontsize=15)
#     plt.show()
    
    return fit

In [10]:
def pred_to_csv(fit, test, x, cols=sample_submission.columns, file="submission.csv", first=False):
    """ Calculate predictions and append to submission csv """
    
    # Recursive prediction
    for i, day in enumerate(np.arange(FIRST, FIRST + LENGTH)):
        test_day = demand_features_eval(test[(test.d <= day) & (test.d >= day - max(LAGS) - max(WINDOWS))])
        test.loc[test.d == day, "demand"] = fit.predict(test_day[x]) # * (1.025 + 0.01 * (day > 1928)) # https://www.kaggle.com/kyakovlev/m5-dark-magic & https://www.kaggle.com/mayer79/m5-forecast-poisson-loss
    
    # Prepare for reshaping
    test = test.assign(id=test.id + "_" + np.where(test.d < FIRST + LENGTH, "validation", "evaluation"),
                       F="F" + (test.d - FIRST + 1 - LENGTH * (test.d >= FIRST + LENGTH)).astype("str"))
    
    # Reshape
    submission = test.pivot(index="id", columns="F", values="demand").reset_index()[cols].fillna(0)
    
    # Export
    submission.to_csv(file, index=False, mode='w' if first else 'a', header=first)
    
    return True

## The loop

For each department id, we will now prepare the model data set (including reshaping of sales data and join of the other data sources) and fit boosted trees individually for this department id. The results are then written to a csv for submission.

In [11]:
for i, dept in enumerate(np.unique(sales.dept_id)):
    start = time.time()
    train, valid, test, x, df = prep_data(sales, 1000, dept) #1150
    fit = fit_model(train, valid, dept, update_hyperparameters=False)
    pred_to_csv(fit, test, x, first=(i==0))
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Time elapsed {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),int(seconds)))
    gc.collect()


Working on dept FOODS_1
{'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmcustomse', 'subsample': 0.5, 'subsample_freq': 1, 'learning_rate': 0.03, 'feature_fraction': 0.5, 'max_bin': 100, 'n_estimators': 1400, 'boost_from_average': False, 'verbose': -1, 'max_depth': 9, 'num_leaves': 1280, 'min_data_in_leaf': 920}
rmse 4.509
{'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmcustomse', 'subsample': 0.5, 'subsample_freq': 1, 'learning_rate': 0.03, 'feature_fraction': 0.5, 'max_bin': 100, 'n_estimators': 1400, 'boost_from_average': False, 'verbose': -1, 'max_depth': 3, 'num_leaves': 1280, 'min_data_in_leaf': 1920}
rmse 4.752
{'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmcustomse', 'subsample': 0.5, 'subsample_freq': 1, 'learning_rate': 0.03, 'feature_fraction': 0.5, 'max_bin': 100, 'n_estimators': 1400, 'boost_from_average': False, 'verbose': -1, 'max_dep

In [12]:
# os.remove("/kaggle/working/params {dept_id}.txt")
# os.remove("/kaggle/working/params_{dept}.txt")
# # os.remove("/kaggle/working/params.txt")
# for i, dept in enumerate(np.unique(sales.dept_id)):
#     os.remove('/kaggle/working/'+dept+'_params2.txt')